In [6]:
import pandas as pd
import numpy as np
df_players = pd.read_csv('c:\\Users\\johnc\\Documents\\Finance and Structures\\nba-chokes-testing\\data\\player_scoring_data.csv')
df_games = pd.read_csv('c:\\Users\\johnc\\Documents\\Finance and Structures\\nba-chokes-testing\\data\\choke_list_full.csv')

In [14]:
df_players = df_players.set_index("game_id")
df_players.head()

,player,points_bf,points_af,team
game_id,,,,
21900837.0,Collins,24.0,11.0,home
21900837.0,Young,12.0,13.0,home
21900837.0,Reddish,18.0,2.0,home
21900837.0,Graham,5.0,0.0,home
21900837.0,Goodwin,2.0,0.0,home


In [15]:
df_games['game_id'] = pd.to_numeric(df_games.game_id)
df_games['matchup'] = df_games.home_team + " vs " + df_games.away_team
df_games = df_games.set_index('game_id')
df_games.head()

,season,home_team,away_team,home_choke,matchup
game_id,,,,,
21900837,2019-20,ATL,DAL,0.0,ATL vs DAL
21900656,2019-20,ATL,LAC,0.0,ATL vs LAC
21900624,2019-20,SAS,ATL,1.0,SAS vs ATL
21900491,2019-20,ORL,ATL,1.0,ORL vs ATL
21900431,2019-20,BKN,ATL,0.0,BKN vs ATL


In [25]:
df_tot = df_players.join(df_games).reset_index()
df_tot.head()

,game_id,player,points_bf,points_af,team,season,home_team,away_team,home_choke,matchup
0,21700002.0,Pachulia,6.0,0.0,home,2017-18,GSW,HOU,1.0,GSW vs HOU
1,21700002.0,Thompson,13.0,3.0,home,2017-18,GSW,HOU,1.0,GSW vs HOU
2,21700002.0,Green,7.0,2.0,home,2017-18,GSW,HOU,1.0,GSW vs HOU
3,21700002.0,Curry,13.0,9.0,home,2017-18,GSW,HOU,1.0,GSW vs HOU
4,21700002.0,McCaw,2.0,2.0,home,2017-18,GSW,HOU,1.0,GSW vs HOU


In [26]:
df_tot.loc[df_tot.team == "home", "team"] = df_tot.home_team
df_tot.loc[df_tot.team == "visitor", "team"] = df_tot.away_team
df_tot = df_tot[["game_id", "matchup", "player", "points_bf", "points_af", "team", "season"]]
df_tot.head()

,game_id,matchup,player,points_bf,points_af,team,season
0,21700002.0,GSW vs HOU,Pachulia,6.0,0.0,GSW,2017-18
1,21700002.0,GSW vs HOU,Thompson,13.0,3.0,GSW,2017-18
2,21700002.0,GSW vs HOU,Green,7.0,2.0,GSW,2017-18
3,21700002.0,GSW vs HOU,Curry,13.0,9.0,GSW,2017-18
4,21700002.0,GSW vs HOU,McCaw,2.0,2.0,GSW,2017-18


In [34]:
df_tot = df_tot[df_tot.game_id > 40000000]
df_tot.shape

(493, 7)

In [42]:
df_tot = pd.melt(df_tot, id_vars=["game_id", "matchup", "player", "team", "season"],
               value_vars=["points_bf", "points_af"],
               var_name="time", value_name="points")

In [44]:
df_tot["time"] = np.select([(df_tot["time"] == "points_bf"), (df_tot["time"] == "points_af")],
                           ["Before Choke/Comeback", "After Choke/Comeback"])
df_tot = df_tot.rename(columns={
    "game_id": "Game ID",
    "matchup": "Matchup",
    "player": "Player",
    "team": "Team",
    "season": "Season",
    "time": "Time",
    "points": "Points"})

In [51]:
d1 = dict()
for game_id in list(np.unique(df_tot["Game ID"])):
    matchup = df_tot.loc[df_tot["Game ID"] == game_id, "Matchup"].iloc[0]
    if matchup not in d1.keys():
        d1[matchup] = df_tot[df_tot["Game ID"] == game_id][["Player", "Team", "Season", "Time", "Points"]].to_dict('records')
    else:
        d1[matchup + ' 2'] = df_tot[df_tot["Game ID"] == game_id][["Player", "Team", "Season", "Time", "Points"]].to_dict('records')   

{'MIA vs PHI': [{'Player': 'Johnson',
   'Team': 'MIA',
   'Season': '2017-18',
   'Time': 'Before Choke/Comeback',
   'Points': 9.0},
  {'Player': 'Dragic',
   'Team': 'MIA',
   'Season': '2017-18',
   'Time': 'Before Choke/Comeback',
   'Points': 20.0},
  {'Player': 'Richardson',
   'Team': 'MIA',
   'Season': '2017-18',
   'Time': 'Before Choke/Comeback',
   'Points': 8.0},
  {'Player': 'Ellington',
   'Team': 'MIA',
   'Season': '2017-18',
   'Time': 'Before Choke/Comeback',
   'Points': 10.0},
  {'Player': 'Wade',
   'Team': 'MIA',
   'Season': '2017-18',
   'Time': 'Before Choke/Comeback',
   'Points': 11.0},
  {'Player': 'Adebayo',
   'Team': 'MIA',
   'Season': '2017-18',
   'Time': 'Before Choke/Comeback',
   'Points': 2.0},
  {'Player': 'Whiteside',
   'Team': 'MIA',
   'Season': '2017-18',
   'Time': 'Before Choke/Comeback',
   'Points': 11.0},
  {'Player': 'Winslow',
   'Team': 'MIA',
   'Season': '2017-18',
   'Time': 'Before Choke/Comeback',
   'Points': 5.0},
  {'Player'

In [53]:
import json
with open('interactive_data.json', 'w') as out:
    json.dump(d1, out)